In [2]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [3]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [4]:
project_name = 'Redisson'
project_owner = 'redisson'
project_name_short = 'redisson'

In [5]:
okhttp_commits_df = pd.read_csv(f'Effort_Estimation_Results_v6/{project_name}.csv')

In [6]:
okhttp_commits_df.head()

,kmean_label,time_taken_mean,time_taken_min,time_taken_max,time_taken_q10,time_taken_q20,time_taken_q25,time_taken_q30,time_taken_q40,time_taken_q50,...,num_dependency_max,num_dependency_std,num_line_affected_mean,num_line_affected_min,num_line_affected_max,num_line_affected_std,actual_num_of_classes_touched_mean,actual_num_of_classes_touched_min,actual_num_of_classes_touched_max,actual_num_of_classes_touched_std


In [294]:
len(okhttp_commits_df)

2

In [295]:
okhttp_commits_df['commit_compared_with']

0    2c24a344a3f8d7e8e9ebd5649237d68844e34a2f
1    d1eb2ece8f9ddb4cc797fd236cc4462682372ef8
Name: commit_compared_with, dtype: object

In [296]:
okhttp_commits_df['num_line_affected_max']

0    30
1    18
Name: num_line_affected_max, dtype: int64

In [297]:
def get_commit_line_changed(sha, project_owner, project_name_short):
    query_url = f"https://api.github.com/repos/{project_owner}/{project_name_short}/commits/{sha}"

    headers = {'Authorization': f'token {github_oauth}'}
    r = requests.get(query_url, headers=headers)
    #print((r.json()))
    return r.json()['stats']['total']

In [298]:
def limit_perc_change(perc):
    if perc > 1:
        return 1
    else:
        return perc

In [299]:
def limit_base_time(time):
    if time < 1:
        return 1
    else:
        return time

In [300]:
get_commit_line_changed('083315a473d8b2fc972a70449272b296683ac38c', 'square', 'okhttp')

358

In [301]:
okhttp_commits_df['commit_line_changed'] = okhttp_commits_df.apply(lambda row: get_commit_line_changed(row['commit_compared_with'], project_owner, project_name_short), axis=1)

In [302]:
okhttp_commits_df['num_line_affected_min']

0    30
1    18
Name: num_line_affected_min, dtype: int64

In [303]:
okhttp_commits_df['commit_line_changed']

0    378
1    232
Name: commit_line_changed, dtype: int64

In [304]:
okhttp_commits_df['refactoring_perc'] = okhttp_commits_df['num_line_affected_min'] / okhttp_commits_df['commit_line_changed']

In [305]:
okhttp_commits_df['refactoring_perc'] = okhttp_commits_df.apply(lambda row: limit_perc_change(row['refactoring_perc']), axis=1)

In [306]:
okhttp_commits_df['refactoring_perc']

0    0.079365
1    0.077586
Name: refactoring_perc, dtype: float64

In [307]:
okhttp_commits_df['refactoring_perc_time_taken'] = okhttp_commits_df['refactoring_perc'] * okhttp_commits_df['time_taken']

In [308]:
okhttp_commits_df['refactoring_perc_time_taken'] = okhttp_commits_df.apply(lambda row: limit_base_time(row['refactoring_perc_time_taken']), axis=1)


In [309]:
okhttp_commits_df['refactoring_perc_time_taken']

0    4.126984
1    1.000000
Name: refactoring_perc_time_taken, dtype: float64

In [ ]:
okhttp_commits_df['cocomoII_time_taken'] = 2.94 * okhttp_commits_df['commit_line_changed'] / 1000 * 176

In [ ]:
okhttp_commits_df['geneticP_time_taken'] = ((1.75992 * okhttp_commits_df['commit_line_changed']/1000) - (4.56 * 0.001 * ((okhttp_commits_df['commit_line_changed']/1000)**2))) * 176

In [ ]:
okhttp_commits_df['softwareMaintenance_time_taken'] = -40 + 6.56*okhttp_commits_df['commit_line_changed']

In [ ]:
okhttp_commits_df['fuzzy_time_taken'] = ( (6*(0.7*okhttp_commits_df['commit_line_changed']/1000)**0.665) + (4*6*(okhttp_commits_df['commit_line_changed']/1000)**0.665) + (6*(1.3*okhttp_commits_df['commit_line_changed']/1000)**0.665) )/(1+4+1)*176

In [310]:
okhttp_commits_df['time_taken']

0    52.0
1     1.0
Name: time_taken, dtype: float64

In [311]:
for column in okhttp_commits_df.columns:
    print(column)

kmean_label
time_taken_mean
time_taken_min
time_taken_max
time_taken_q10
time_taken_q20
time_taken_q25
time_taken_q30
time_taken_q40
time_taken_q50
time_taken_q60
time_taken_q70
time_taken_q75
time_taken_q80
time_taken_q90
sha
name
email
date
login
message
parent_sha
parent_date
time_taken
contains_refactoring
project_name
commit_compared_with
cbo_mean
cbo_min
cbo_max
cbo_std
wmc_mean
wmc_min
wmc_max
wmc_std
dit_mean
dit_min
dit_max
dit_std
rfc_mean
rfc_min
rfc_max
rfc_std
lcom_mean
lcom_min
lcom_max
lcom_std
totalMethods_mean
totalMethods_min
totalMethods_max
totalMethods_std
staticMethods_mean
staticMethods_min
staticMethods_max
staticMethods_std
publicMethods_mean
publicMethods_min
publicMethods_max
publicMethods_std
privateMethods_mean
privateMethods_min
privateMethods_max
privateMethods_std
protectedMethods_mean
protectedMethods_min
protectedMethods_max
protectedMethods_std
defaultMethods_mean
defaultMethods_min
defaultMethods_max
defaultMethods_std
abstractMethods_mean
abstractMe

In [312]:
okhttp_commits_df.to_csv(f'Effort_Estimation_Results_3E_v2/{project_name}.csv', index=False)

In [14]:
## Test formula
loc = 30
kloc = loc/1000
cocomo = 2.94 * kloc * 176
geneticP = ((1.75992 * kloc) - (4.56 * 0.001 * (kloc**2))) * 176
testgeneticP = ((1.75992 * loc/1000) - (4.56 * 0.001 * ((loc/1000)**2))) * 176
softwareMaintenance = -40 + 6.56*loc
fuzzy = ( (6*(0.7*kloc)**0.665) + (4*6*kloc**0.665) + (6*(1.3*kloc)**0.665) )/(1+4+1)*176
testfuzzy = ( (6*(0.7*loc/1000)**0.665) + (4*6*(loc/1000)**0.665) + (6*(1.3*loc/1000)**0.665) )/(1+4+1)*176



print("Cocomo: " + str(cocomo))
print("GeneticP: " + str(geneticP))
print("softwareMaintenance: " + str(softwareMaintenance))
print("fuzzy: " + str(fuzzy))

print("test GeneticP: " + str(testgeneticP))
print("test testfuzzy: " + str(testfuzzy))



Cocomo: 15.5232
GeneticP: 9.291655295999998
softwareMaintenance: 156.79999999999998
fuzzy: 102.20220527519842
test GeneticP: 9.291655295999998
test testfuzzy: 102.20220527519842


In [15]:
# Cocomo formula = 2.94 * kloc * 176

In [ ]:
# GeneticP formula = 1.75992 * kloc - 4.56 * 0.001 * kloc^2

